# Предсказание стоимости жилья

В проекте вам нужно обучить модель линейной регрессии на данных о жилье в Калифорнии в 1990 году. На основе данных нужно предсказать медианную стоимость дома в жилом массиве. Обучите модель и сделайте предсказания на тестовой выборке. Для оценки качества модели используйте метрики RMSE, MAE и R2.

**Описание проекта:** предоставлены данных о жилье в Калифорнии в 1990 году. На основе данных нужно предсказать медианную стоимость дома в жилом массиве.

**Цель исследования:** Необходимо редсказать медианную стоимость дома в жилом массиве.

**Ход исследования:**

- Обучение модели на подготовленных данных;
- Сделать предсказания на тестовой выборке;
- Оценить качество модели, используя RMSE, MAE и R2.

**Общий вывод:** резюмирование полуенных результатов, формулировка ключевых выводов и результатов.

In [1]:
import pandas as pd 
import numpy as np
import pyspark

from pyspark.sql import SparkSession
from pyspark.sql.types import *
import pyspark.sql.functions as F
import seaborn as sns
from pyspark.ml.feature import OneHotEncoder 
from pyspark.ml.feature import StringIndexer, VectorAssembler, StandardScaler
from pyspark.ml.regression import LinearRegression
from pyspark.ml.evaluation import RegressionEvaluator
RANDOM_SEED = 2022

## Подготовка данных

### Инициализируем локальную Spark-сессию

In [2]:
spark = SparkSession.builder \
                    .master("local") \
                    .appName("EDA California Housing") \
                    .getOrCreate()

### Прочитаем содержимое файла

In [3]:
df_housing = spark.read.option('header', 'true').csv('/datasets/housing.csv', inferSchema = True) 
df_housing.printSchema() 

root
 |-- longitude: double (nullable = true)
 |-- latitude: double (nullable = true)
 |-- housing_median_age: double (nullable = true)
 |-- total_rooms: double (nullable = true)
 |-- total_bedrooms: double (nullable = true)
 |-- population: double (nullable = true)
 |-- households: double (nullable = true)
 |-- median_income: double (nullable = true)
 |-- median_house_value: double (nullable = true)
 |-- ocean_proximity: string (nullable = true)



### Выведем типы данных колонок датасета

In [4]:
print(pd.DataFrame(df_housing.dtypes, columns=['column', 'type']))

               column    type
0           longitude  double
1            latitude  double
2  housing_median_age  double
3         total_rooms  double
4      total_bedrooms  double
5          population  double
6          households  double
7       median_income  double
8  median_house_value  double
9     ocean_proximity  string


**Вывод**: в исследуемом датасете только одна колонка с категоральными данными. Все остальные с численными.

### Выполним предобработку данных

Более детально изучим данные. Для этого вызовем метод describe.

In [5]:
df_housing.describe().toPandas()

,summary,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,count,20640,20640,20640,20640,20433,20640,20640,20640,20640,20640
1,mean,-119.56970445736148,35.6318614341087,28.639486434108527,2635.7630813953488,537.8705525375618,1425.4767441860465,499.5396802325581,3.8706710029070246,206855.81690891474,None
2,stddev,2.003531723502584,2.135952397457101,12.58555761211163,2181.6152515827944,421.38507007403115,1132.46212176534,382.3297528316098,1.899821717945263,115395.61587441359,None
3,min,-124.35,32.54,1.0,2.0,1.0,3.0,1.0,0.4999,14999.0,<1H OCEAN
4,max,-114.31,41.95,52.0,39320.0,6445.0,35682.0,6082.0,15.0001,500001.0,NEAR OCEAN


Из данных видно следующее:
1. Значения координат корректные, т.к. минимальные и максимальные значения долготы и широты соответветствуют Калифорнии;
2. Медианный возраст жителей массива корректный;
3. Общее количество спален имеет пропуски. Количество пропусков 207. Заполнить пропуски можно нулевыми значениями.

#### Заполним пропуски в данных

In [6]:
df_housing = df_housing.na.fill(0)

Проверка.

In [7]:
columns = df_housing.columns


for column in columns:
    print(column, df_housing.where(F.isnan(column) | F.col(column).isNull()).count())

longitude 0
latitude 0
housing_median_age 0
total_rooms 0
total_bedrooms 0
population 0
households 0
median_income 0
median_house_value 0
ocean_proximity 0


Проверка выполнена. Пропусков не осталось. Теперь разделим колонки на два типа: числовые и категориальные. 

In [8]:
categorical_cols = ['ocean_proximity']
numerical_cols  = ["longitude", "latitude", "housing_median_age", "total_rooms", "total_bedrooms", "population",
                   "households", "median_income"]
target = "median_house_value" 

#### Преобразуем колонку с категориальными данными с помощью ОНЕ

In [9]:
#Разделим модель на тренировочную и тестовую выборки:
train_data, test_data = df_housing.randomSplit([.8,.2], seed=RANDOM_SEED)
print(train_data.count(), test_data.count()) 

16418 4222


In [10]:
#Сначала трансформируем категориальные признаки в числовое представление с помощью трансформера StringIndexer:
indexer = StringIndexer(inputCols=categorical_cols, 
                        outputCols=[c+'_idx' for c in categorical_cols], handleInvalid="keep") 
model = indexer.fit(train_data)
train_data = model.transform(train_data)
test_data = model.transform(test_data)

In [11]:
#Далее выполним OHE-кодирование для категорий:
ohe = OneHotEncoder(inputCols=[c+'_idx' for c in categorical_cols],
                        outputCols=[c+'_ohe' for c in categorical_cols], handleInvalid="keep")
model = ohe.fit(train_data)
train_data = model.transform(train_data)
test_data = model.transform(test_data)
train_data.describe().toPandas()

,summary,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity,ocean_proximity_idx
0,count,16418,16418,16418,16418,16418,16418,16418,16418,16418,16418,16418
1,mean,-119.56720855158979,35.632537458886816,28.67809721037885,2635.865878913388,531.5313680107199,1424.8680107199416,499.401936898526,3.8599032647094864,206166.77207942502,None,0.9091850408088683
2,stddev,2.007170358524894,2.137638120554967,12.583767547384063,2194.1395066716864,422.6201945683686,1141.8443927369387,383.6679584089657,1.8958698224587758,115137.08168148558,None,1.0036054542951498
3,min,-124.35,32.54,1.0,2.0,0.0,3.0,1.0,0.4999,14999.0,<1H OCEAN,0.0
4,max,-114.31,41.95,52.0,39320.0,6445.0,35682.0,6082.0,15.0001,500001.0,NEAR OCEAN,3.0


In [12]:
test_data.describe().toPandas()

,summary,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity,ocean_proximity_idx
0,count,4222,4222,4222,4222,4222,4222,4222,4222,4222,4222,4222
1,mean,-119.57941023211727,35.62923259118911,28.489341544291804,2635.363334912364,536.1504026527712,1427.8439128375178,500.07531975367124,3.912543273330184,209535.29062055898,None,0.9071530080530554
2,stddev,1.9895257132603632,2.1296350603031557,12.592881835143269,2132.4688640647682,422.93434838114814,1095.3425940223842,377.12527133454745,1.914762150871446,116370.4567235921,None,1.0083287346111869
3,min,-124.3,32.55,1.0,11.0,0.0,13.0,5.0,0.4999,17500.0,<1H OCEAN,0.0
4,max,-114.47,41.84,52.0,21533.0,4492.0,12427.0,4372.0,15.0001,500001.0,NEAR OCEAN,4.0


In [13]:
#Объединим признаки в один вектор с помощью VectorAssembler:
categorical_assembler = \
        VectorAssembler(inputCols=[c+'_ohe' for c in categorical_cols],
                                        outputCol="categorical_features", handleInvalid="keep")
train_data = categorical_assembler.transform(train_data)
test_data = categorical_assembler.transform(test_data)

**Выводы по разделу**:
1. Пропущенные значения были только в колонке с общим количеством спален. Скорее всего, в жилье с одной комнатой их количество не указывалось;
2. Колонка с категориальным признаком была трансформирована с помощью техники ОНЕ.

## Обучение моделей

In [14]:
#Прежде, чем перейти к обучению моделей мы трансформируем и числовые признаки. Это нужно, чтобы сильные выбросы не 
#влияли на результат работы модели.
numerical_assembler = VectorAssembler(inputCols=numerical_cols, outputCol="numerical_features")
train_data = numerical_assembler.transform(train_data)
test_data = numerical_assembler.transform(test_data)
standardScaler = StandardScaler(inputCol='numerical_features',outputCol="numerical_features_scaled")
model = standardScaler.fit(train_data)
train_data = model.transform(train_data)
test_data = model.transform(test_data)

In [15]:
#Далее соберем полученные категорийные и числовые признаки с помощью VectorAssembler:
all_features = ['categorical_features','numerical_features_scaled']

final_assembler = VectorAssembler(inputCols=all_features, outputCol="features") 
train_data = final_assembler.transform(train_data)
test_data = final_assembler.transform(test_data)
train_data.select(all_features).show(3)
test_data.select(all_features).show(3)

+--------------------+-------------------------+
|categorical_features|numerical_features_scaled|
+--------------------+-------------------------+
|       (5,[2],[1.0])|     [-61.952887791441...|
|       (5,[2],[1.0])|     [-61.927977100733...|
|       (5,[2],[1.0])|     [-61.913030686308...|
+--------------------+-------------------------+
only showing top 3 rows

+--------------------+-------------------------+
|categorical_features|numerical_features_scaled|
+--------------------+-------------------------+
|       (5,[2],[1.0])|     [-61.927977100733...|
|       (5,[2],[1.0])|     [-61.893102133741...|
|       (5,[2],[1.0])|     [-61.883137857458...|
+--------------------+-------------------------+
only showing top 3 rows



### Обучение модели на всех данных

In [16]:
lr_1 = LinearRegression(labelCol=target, featuresCol='features',regParam=0.0)

In [17]:
model = lr_1.fit(train_data) 

24/12/18 05:33:35 WARN Instrumentation: [1ed6a7bd] regParam is zero, which might cause numerical instability and overfitting.
24/12/18 05:33:35 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeSystemBLAS
24/12/18 05:33:35 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeRefBLAS
24/12/18 05:33:36 WARN LAPACK: Failed to load implementation from: com.github.fommil.netlib.NativeSystemLAPACK
24/12/18 05:33:36 WARN LAPACK: Failed to load implementation from: com.github.fommil.netlib.NativeRefLAPACK
24/12/18 05:33:36 WARN Instrumentation: [1ed6a7bd] Cholesky solver failed due to singular covariance matrix. Retrying with Quasi-Newton solver.


In [18]:
#Сохраним трансформированную таблицу с колонкой предсказания первой модели в переменной predictions:
predictions = model.transform(test_data)
predictions_col = 'prediction'

### Обучение модели только на числовых данных

In [19]:
lr_2 = LinearRegression(labelCol=target, featuresCol='numerical_features_scaled', regParam=0.0)

In [20]:
model2 = lr_2.fit(train_data) 

24/12/18 05:33:39 WARN Instrumentation: [b98646c9] regParam is zero, which might cause numerical instability and overfitting.


In [21]:
#Сохраним трансформированную таблицу с колонкой предсказания второй модели в переменной predictions2:
predictions2 = model2.transform(test_data)

## Анализ результатов

In [22]:
#Для анализа результатов работы модели создадим объект класса RegressionEvaluator:
evaluator = RegressionEvaluator(predictionCol=predictions_col, labelCol = target)

### RMSE

Первая метрика - корень от средней квадратической ошибки. В свою очередь средняя квадратическая ошибка - это величина равная сумме квадратов отклонений (разница между предсказанием и фактической величиной целевого признака), разделенной на количество объектов. Чем точнее модель, тем больше значение RMSE будет стремиться к нулю.

In [23]:
#Для первой (учитывающей все признаки):
evaluator.evaluate(predictions, {evaluator.metricName: "rmse"})

68547.35345527249

In [24]:
#И для второй (учитывающей только количественные признаки):
evaluator.evaluate(predictions2, {evaluator.metricName: "rmse"})

69278.38973957153

**Вывод**: первая модель работает точнее.

### MAE

Среднее абсолютное значение или MAE - это величина, равная сумме всех модулей отклонений (разница между предсказанием и фактическим значением целевого признака), разделенная на количество объектов в выборке.

Соответственно, для модели, которая идеально работает, MAE будет равен 0. Поэтому, чем меньше будет значение этой метрики, тем точнее модель.

In [25]:
#Для первой (учитывающей все признаки):
evaluator.evaluate(predictions, {evaluator.metricName: "mae"})

49977.32492934058

In [26]:
#И для второй (учитывающей только количественные признаки):
evaluator.evaluate(predictions2, {evaluator.metricName: "mae"})

51010.980939849156

**Вывод**: первая модель работает точнее.

### R2

Коэффициент детерминации (R2) равен значению равному:

1 - доля среднеквадратической ошибки модели от MSE среднего.

Это значит, что R2 равно единице, когда модель идеально предсказывает результат, а 0 - когда работает также, как и среднее. Соответственно, чем выше R2, тем точнее модель.

In [27]:
#Для первой (учитывающей все признаки):
evaluator.evaluate(predictions, {evaluator.metricName: "r2"})

0.652944618740229

In [28]:
#И для второй (учитывающей только количественные признаки):
evaluator.evaluate(predictions2, {evaluator.metricName: "r2"})

0.645502669979857

**Вывод**: первая модель работает точнее.

# Выводы:

В результате выполнения данной работы получено следующее:

- Инициализирована локальная Spark-сессию;
- Проведена предобработка данных, используя методы pySpark;
- Проведено исследование данных на наличие пропусков и их устранение;
- Преобразована колонка с категориальными значениями техникой One hot encoding;
- Проведена стандартизации количественных признаков.

При построение двух моделей линейной регрессии на разных наборах данных:

- используя все данные из файла;
- используя только числовые переменные, исключив категориальные.

Согласно всем трем метрикам RMSE, MAE и R2 точнее работает первая модель, использующая все данные из файла.

In [29]:
spark.stop()